## A simple data munger

### Uses selenium and chromedriver to open a webpage and read off fluorescence absorption and emission values for all fluorophores and dyes available on that page.
### The read values are then written to a Excel file for using in further calculations for overlap and filter transmission calculations.

In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver=webdriver.Chrome("C:\\Users\\Amarb\\Downloads\\chromedriver_win32\\chromedriver.exe")

In [3]:
sleep(0.2)
driver.get("http://www.qpcrdesign.com/spectral-overlay")

In [4]:
a1=driver.find_elements_by_xpath('//*[@class="container-fluid"]')
inp_list=a1[1].find_elements_by_class_name('checkbox-inline')
spec_dict={};
for inp_elem in inp_list:
    (inp_elem.find_element_by_tag_name('input')).click()
    sleep(0.5)
    ab_tb=driver.find_element_by_xpath('//*[@id="absorption-spec"]/div/div[1]/div/div/table')
    em_tb=driver.find_element_by_xpath('//*[@id="emission-spec"]/div/div[1]/div/div/table')
    ab_entries=ab_tb.find_elements_by_tag_name('td')
    ab_spec_list=[];
    for entry in ab_entries:
        ab_tb_val=entry.get_attribute('innerHTML')
        if len(ab_tb_val)>0:
            ab_spec_list.append(float(ab_tb_val))
        else:
            ab_spec_list.append(0.0)
    
    em_entries=em_tb.find_elements_by_tag_name('td')
    em_spec_list=[];
    for entry in em_entries:
        em_tb_val=entry.get_attribute('innerHTML')
        if len(em_tb_val)>0:
            em_spec_list.append(float(em_tb_val))
        else:
            em_spec_list.append(0.0)
    (inp_elem.find_element_by_tag_name('input')).click()
    spec_dict.update({inp_elem.find_element_by_tag_name('input').get_attribute('value') : [ab_spec_list[::2],ab_spec_list[1::2],em_spec_list[1::2] ]})

In [5]:
driver.close()

In [6]:
import xlsxwriter

In [7]:
workbook = xlsxwriter.Workbook('flou2.xlsx')
for ky in spec_dict.keys():
    worksheet = workbook.add_worksheet(ky)
    worksheet.write('A1','Wavelength')
    worksheet.write('B1','Absorption')
    worksheet.write('C1','Emission')
    row=1
    col=0
    for dat in spec_dict[ky][0]:
        worksheet.write_number(row, col, dat)
        row += 1
    row=1
    for absp in spec_dict[ky][1]:
        worksheet.write_number(row, col + 1, absp)
        row += 1
    row=1
    for emis in spec_dict[ky][2]:
        worksheet.write_number(row, col + 2, emis)
        row += 1

In [8]:
workbook.close()